In [6]:
import pandas as pd
import numpy as np

!conda install -c anaconda beautifulsoup4 

Solving environment: done

# All requested packages already installed.



In [7]:
import requests
from bs4 import BeautifulSoup

In [8]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source, 'html5lib')
my_table = soup.find('table',{'class':'wikitable sortable'})
trs=my_table.find_all('tr')
ths = my_table.find_all('th')
headings = [th.text.strip() for th in ths]
headings

postal_codes_dict = {} # initialize an empty dictionary to save the data in
for tr in my_table.find_all('tr'):
    try:
        tds = tr.find_all('td')
        if not tds:
            continue
        postal_code, borough, neighborhood = [td.text.strip() for td in tds[:3]]
        #print('; '.join([postal_code, borough, neighborhood]))
        if borough == 'Not assigned':
            neighborhoods = []
        else:
            postal_codes_dict[postal_code] = {}
            postal_codes_dict[postal_code]['borough'] = borough
            if neighborhood == 'Not assigned':
                neighborhood = borough
            postal_codes_dict[postal_code]['neighborhoods'] = neighborhood
    except:
        pass
postal_codes_dict
# create an empty dataframe
columns = ['PostalCode', 'Borough', 'Neighborhood']
toronto_data = pd.DataFrame(columns=columns)
toronto_data

# populate dataframe with data from dictionary
for ind, postal_code in enumerate(postal_codes_dict):
    borough = postal_codes_dict[postal_code]['borough']
    neighborhood = postal_codes_dict[postal_code]['neighborhoods']
    toronto_data = toronto_data.append({"PostalCode": postal_code, 
                                        "Borough": borough, 
                                        "Neighborhood": neighborhood},
                                        ignore_index=True)

df2=toronto_data.groupby(['PostalCode','Borough'])['Neighborhood'].apply(lambda x: ", ".join(x.astype(str))).reset_index()
#df2
df2.shape

(103, 3)